### Задача 1 (10 баллов). 

Представьте, что вы разрабатываете игру, в которой игрок управляет флотом космических кораблей. Каждый корабль имеет свои характеристики, вооружение и экипаж. Чтобы управлять флотом, игрок должен делегировать различные задачи (например, атака, оборона, ремонт) разным кораблям и экипажу. Ваша задача — создать систему классов, которая использует композицию для организации кораблей, экипажа и вооружения, а также делегирует задачи между этими объектами.

Требования:

Создайте класс Ship, который представляет космический корабль. Корабль должен:

- Иметь имя, тип корабля (например, "battlecruiser", "frigate", "destroyer") и уровень прочности (например, 100%).
- Иметь экипаж, состоящий из пилота и инженера (с помощью композиции).
- Иметь вооружение (также через композицию), которое будет делегироваться классу Weapon.
- Мог выполнять действия атаки, ремонта и защиты, но делегировать эти задачи соответствующим объектам (например, пилоту или инженеру).

Создайте класс CrewMember (член экипажа), который может выполнять различные задачи:

- Pilot — отвечает за управление кораблём, атаки и манёвры.
- Engineer — отвечает за ремонт и восстановление прочности корабля.

Используйте композицию для того, чтобы корабль "имел" пилота и инженера.

Создайте класс Weapon, который будет представлять вооружение корабля:

- Каждый корабль может иметь одно или несколько оружий.
- Оружие может атаковать противника, но уровень атаки зависит от типа оружия и его состояния (например, лазер, ракеты, плазменные пушки).

Используйте композицию для добавления оружия к кораблю и делегируйте задачи атаки объектам класса Weapon.

Класс Fleet должен представлять целый флот кораблей и управлять их действиями:

- Возможность добавлять корабли во флот.
- Делегирование задач атаки и ремонта флоту, который будет распределять их между кораблями.

Пример использования:


#### Определение оружия

    laser = Weapon("Laser Cannon", 50)
    missile = Weapon("Missile Launcher", 100)

#### Создание экипажа

    pilot = Pilot("John Doe")
    engineer = Engineer("Jane Smith")

#### Создание кораблей

    ship1 = Ship("USS Enterprise", "battlecruiser", pilot, engineer)
    ship2 = Ship("Falcon", "frigate", Pilot("Han Solo"), Engineer("Chewbacca"))

#### Добавление вооружения к кораблям

    ship1.add_weapon(laser)
    ship2.add_weapon(missile)

#### Создание флота

    fleet = Fleet()
    fleet.add_ship(ship1)
    fleet.add_ship(ship2)

#### Атака флотом

    print("Флот атакует!")
    fleet.attack_all()

#### Ремонт флота

    print("\nФлот выполняет ремонт!")
    fleet.repair_all()

#### Результат:

    # USS Enterprise атакует с помощью Laser Cannon (урон 50)
    # Falcon атакует с помощью Missile Launcher (урон 100)
    # USS Enterprise был отремонтирован инженером Jane Smith до полной прочности.
    # Falcon был отремонтирован инженером Chewbacca до полной прочности.
    
Подсказки:

Композиция: Класс Ship должен содержать объекты экипажа и вооружения, а класс Fleet должен содержать объекты кораблей.

Делегирование: Методы атаки, защиты и ремонта должны вызывать методы у соответствующих объектов. Например, при вызове метода attack() у корабля, этот метод должен делегировать выполнение атаки объекту Weapon и экипажу (пилоту).

Взаимодействие классов: Корабль не выполняет все задачи сам, он делегирует их своим компонентам (экипажу и оружию).

In [43]:
from time import sleep

class Ship:
    def __init__(self, name, resistance : int):
        self.name = name
        self.resistance = resistance
        self.weapon = []

    def __str__(self):
        return f"Корабль {self.name}, уровень прочности {self.resistance}"

    def get_a_crew(self):
        pilot_name = input('Выберите имя пилота')
        engineer_name = input('Выберите имя инженера')
        self.crew = Crew(self, pilot_name, engineer_name)
        return self.crew
    
    def get_weapon(self):
        w_name = input('Введите название оружия')
        w_type = input('Введите вид оружия (laser/missile/gun)')
        w_level = input('Введите уровень оружия (число)')  
        w = Weapon(w_name, w_type, w_level)
        self.weapon.append(w)

    def show_the_weapon(self):
        for w in self.weapon:
            print(self.weapon.index(w) + 1, w)

    def show_the_crew(self):
        print(f"Инженер {self.crew.pilot.name}, пилот {self.crew.engineer.name}")

    def attack(self):
        self.show_the_weapon()
        sleep(1) #иногда список оружия выводится уже после окна ввода, добавила задержку на секунду, чтобы такого не было
        w = int(input('Введите номер оружия, которым хотите воспользоваться')) - 1
        self.crew.pilot.attack(self.weapon[w])
    
    def repair(self):
        obj = input('Что Вы хотите отремонтировать: 1 -- корабль, 2 -- оружие.')
        if obj == '1':
            self.crew.repair_ship()
        else:
            self.crew.repair_weapon()



In [42]:


class Pilot:
    def __init__(self, name, ship):
        self.name = name
        self.ship = ship

    def __repr__(self):
        return f"Пилот {self.name}, в команде корабля {self.ship.name}"

    def attack(self, weapon):
        print(f"{self.ship.name} атакует оружием {weapon.name}")
        weapon.fire()

     

class Engineer:
     def __init__(self, name, ship):
        self.name = name
        self.ship = ship

     def __repr__(self):
        return f"Инженер {self.name}, в команде корабля {self.ship.name}"


     def repair(self, obj, t):
        sleep(t)
        obj.resistance += t
        print(f"{obj.name} починен, уровень прочности: {obj.resistance}")



class Crew:
     def __init__(self, ship, pilot_name, engineer_name):
          self.ship = ship
          self.pilot = Pilot(pilot_name, self.ship)
          self.engineer = Engineer(engineer_name, self.ship)

     def __repr__(self):
          return f"Команда корабля {self.ship.name}: инженер {self.engineer.name}, пилот {self.pilot.name}"
     

     def repair_ship(self):
          t = int(input('Насколько серьезен урон? (Введите число)'))
          self.engineer.repair(self.ship, t)
          

     def repair_weapon(self):
         self.ship.show_the_weapon()
         w = int(input('Введите номер оружия, которое хотите отремонтировать')) - 1
         t = int(input('Насколько серьезен урон? (Введите число)'))
         self.engineer.repair(self.ship.weapon[w], t)

         

     

In [34]:
class Weapon:
    def __init__(self, name, type, level:int): #types: laser, missile, gun
        self.name = name
        self.type = type
        if self.type == 'laser':
            self.coef = 3
            self.resistance = 10
        elif self.type == 'missile':
            self.coef = 4
            self.resistance = 7
        else:
            self.coef = 2
            self.resistance = 8
        self.level = int(level) * self.coef

    def __str__(self):
        return f"{self.name}, класс {self.type}, уровень {self.level}"

    def fire(self):
        self.resistance -= 1
        print(f'Противнику нанесен урон {self.level}') 

    


In [58]:
class Fleet:
    def __init__(self):
        self.ships = []

    def add_a_ship(self):
        name = input('Введите название корабля')
        res = input('Введите уровень прочности корабля')
        s = Ship(name, res) #создаем корабль и сразу добавляем ему команду и оружие
        s.get_a_crew()
        n = int(input('Сколько оружия Вы хотите добавить на этот корабль?'))
        for x in range(n):
            s.get_weapon()
        self.ships.append(s)
        print(f"{s} добавлен.\nКоманда:")
        s.show_the_crew()
        print('Оружие:')
        s.show_the_weapon()
        

    def show_the_ships(self):
        for s in self.ships:
            print(self.ships.index(s) + 1, s)

    def show_the_ships_full(self):
        for s in self.ships:
            print(s)
            s.show_the_crew()
            s.show_the_weapon()

    

    def choose_a_ship(self):
        self.show_the_ships()
        sleep(1)
        s = int(input('Введите номер корабля')) - 1
        return self.ships[s]

    def add_weapon_to_ship(self): #для добавления оружия в уже созданный корабль
        s = self.choose_a_ship()
        n = int(input('Сколько оружия Вы хотите добавить на этот корабль?'))
        for x in range(n):
            s.get_weapon()
        s.show_the_weapon()

    def attack(self):
        self.choose_a_ship().attack()

    def repair(self):
        self.choose_a_ship().repair()



    

In [59]:
f = Fleet()

In [62]:
f.add_a_ship()

Корабль Falcon, уровень прочности 80 добавлен.
Команда:
Инженер Рик, пилот Морти
Оружие:
1 Missile Launcher, класс missile, уровень 36
2 Pukalka, класс gun, уровень 6


In [63]:
f.show_the_ships()

1 Корабль USS Enterprise, уровень прочности 100
2 Корабль Falcon, уровень прочности 80


In [64]:
f.attack()

1 Корабль USS Enterprise, уровень прочности 100
2 Корабль Falcon, уровень прочности 80
1 Missile Launcher, класс missile, уровень 36
2 Pukalka, класс gun, уровень 6
Falcon атакует оружием Pukalka
Противнику нанесен урон 6


In [65]:
f.repair()

1 Корабль USS Enterprise, уровень прочности 100
2 Корабль Falcon, уровень прочности 80
1 Missile Launcher, класс missile, уровень 36
2 Pukalka, класс gun, уровень 6
Pukalka починен, уровень прочности: 10
